In [1]:
# conda activate WeatherModel

from bids.layout import BIDSLayout

import numpy as np

import pandas as pd

from nilearn.glm.first_level import FirstLevelModel

from nilearn.plotting import plot_design_matrix

from nilearn.plotting import plot_stat_map, plot_glass_brain

from nilearn.glm.thresholding import threshold_stats_img

import matplotlib.pyplot as plt

import warnings
from nilearn import plotting
from nilearn.reporting import get_clusters_table

# Avoid getting warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
import os
import sys
    

In [2]:
# fixed variables 

ds_path = '/Users/jonathantsay/GestureContextPilot/CodeForExp/JT_Semantomotor/data/bids'

# Initialize the BIDS layout and include the derivatives in it
layout = BIDSLayout(ds_path, derivatives = True)
layout.add_derivatives(os.path.join(ds_path, "results", "first-level"))

TR = layout.get_tr()
confounds_of_interest = ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']



In [3]:

sID = "04"
runs = [0, 1, 2, 3, 4]

bold = layout.get(subject=sID, datatype='func', space='T1w', desc='preproc', extension='.nii.gz', \
                 return_type='filename')
events = layout.get(subject=sID, datatype='func', suffix='events', extension=".tsv", return_type='filename')
confounds = layout.get(subject=sID, datatype='func', desc='confounds', extension=".tsv", return_type='filename')


for run in runs:
    
    confounds_glm = []
    this_conf = pd.read_table(confounds[run])
    conf_subset = this_conf[confounds_of_interest].fillna(0) # replace NaN with 0
    confounds_glm.append(conf_subset)

    fmri_glm = FirstLevelModel(
        t_r = TR,
        slice_time_ref = TR/2,
        hrf_model = 'spm',
        drift_model = 'Cosine',
        high_pass = 1./128,
        smoothing_fwhm = 2,
        noise_model = 'ar1'
    )

    fmri_glm = fmri_glm.fit(bold[run], events[run], confounds_glm)
    
    contrast_list = []
    design_matrices = fmri_glm.design_matrices_
    
    for design_matrix in design_matrices:
    
        """A small routine to append zeros in contrast vectors"""
        n_columns = design_matrix.shape[1] #number of predictors in our model
        def pad_vector(contrast_, n_columns):    
            return np.hstack((contrast_, np.zeros(n_columns - len(contrast_))))
    
        contrasts = {'L1': pad_vector([1, 0, 0, 0, 
                                   0, 0, 0, 0, 
                                   0, 0, 0, 0], n_columns),
                 'N1': pad_vector([0, 1, 0, 0, 
                                   0, 0, 0, 0, 
                                   0, 0, 0, 0], n_columns),
                 'Ac1': pad_vector([0, 0, 1, 0, 
                                   0, 0, 0, 0, 
                                   0, 0, 0, 0], n_columns),
                 'Ab1': pad_vector([0, 0, 0, 1, 
                                   0, 0, 0, 0, 
                                   0, 0, 0, 0], n_columns),
                 'L2': pad_vector([0, 0, 0, 0, 
                                   1, 0, 0, 0, 
                                   0, 0, 0, 0], n_columns),
                 'N2': pad_vector([0, 0, 0, 0, 
                                   0, 1, 0, 0, 
                                   0, 0, 0, 0], n_columns),
                 'Ac2': pad_vector([0, 0, 0, 0, 
                                   0, 0, 1, 0, 
                                   0, 0, 0, 0], n_columns),
                 'Ab2': pad_vector([0, 0, 0, 0, 
                                   0, 0, 0, 1, 
                                   0, 0, 0, 0], n_columns),
                 'L3': pad_vector([0, 0, 0, 0, 
                                   0, 0, 0, 0, 
                                   1, 0, 0, 0], n_columns),
                 'N3': pad_vector([0, 0, 0, 0, 
                                   0, 0, 0, 0, 
                                   0, 1, 0, 0], n_columns),
                 'Ac3': pad_vector([0, 0, 0, 0, 
                                   0, 0, 0, 0, 
                                   0, 0, 1, 0], n_columns),
                 'Ab3': pad_vector([0, 0, 0, 0, 
                                   0, 0, 0, 0, 
                                   0, 0, 0, 1], n_columns),
                 'move': pad_vector([1, 1, 1, 1, 
                                     1, 1, 1, 1, 
                                     1, 1, 1, 1], n_columns), 
                 'palm': pad_vector([1, 1, 1, 1, 
                                     0, 0, 0, 0, 
                                     0, 0, 0, 0], n_columns),
                 'fist': pad_vector([0, 0, 0, 0, 
                                     1, 1, 1, 1, 
                                     0, 0, 0, 0], n_columns),
                 'peace': pad_vector([0, 0, 0, 0, 
                                      0, 0, 0, 0, 
                                     1, 1, 1, 1], n_columns), 
                 'letter': pad_vector([1, 0, 0, 0, 
                                       1, 0, 0, 0, 
                                       1, 0, 0, 0], n_columns), 
                 'number': pad_vector([0, 1, 0, 0, 
                                       0, 1, 0, 0, 
                                       0, 1, 0, 0], n_columns), 
                 'action': pad_vector([0, 0, 1, 0, 
                                       0, 0, 1, 0, 
                                       0, 0, 1, 0], n_columns), 
                 'abstract': pad_vector([0, 0, 0, 1, 
                                        0, 0, 0, 1, 
                                       0, 0, 0, 1], n_columns),
                  'concrete': pad_vector([-1, -1, 1, 1, 
                                       -1, -1, 1, 1, 
                                       -1, -1, 1, 1], n_columns)}
    
        contrast_list.append(contrasts)
        
        model_name = 'first-level'

        outdir = os.path.join(ds_path, 'results', model_name, 'sub-' + sID)
        if not os.path.exists(outdir):
            os.makedirs(outdir)
    
        stats_type = ['effect_size'] #['z_score', 'stat']
        
        for stats in stats_type:
            for contrast_id in contrast_list[0].keys():    
                stats_map = fmri_glm.compute_contrast(
                [c[contrast_id] for c in contrast_list], 
                output_type = stats)
                # Save results following BIDS standart
                res_name = os.path.basename(bold[0]).split("run")[0]
                stats_map.to_filename(os.path.join(outdir, res_name + 'run-0' + str(run + 1) + '_' + 'desc-' + contrast_id + '_' + stats + '.nii.gz'))
    
    print(os.path.join(outdir, res_name + 'run-0' + str(run + 1) + '_' + 'desc-' + contrast_id + '_' + stats + '.nii.gz'))
    

/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'jitter.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'ms_txt' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'trial_fixation.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'gesture_image' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/

/Users/jonathantsay/GestureContextPilot/CodeForExp/JT_Semantomotor/data/bids/results/first-level/sub-04/sub-04_task-gesture_run-01_desc-concrete_effect_size.nii.gz


/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'jitter.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'ms_txt' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'trial_fixation.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'gesture_image' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/

/Users/jonathantsay/GestureContextPilot/CodeForExp/JT_Semantomotor/data/bids/results/first-level/sub-04/sub-04_task-gesture_run-02_desc-concrete_effect_size.nii.gz


/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'jitter.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'ms_txt' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'trial_fixation.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'gesture_image' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/

/Users/jonathantsay/GestureContextPilot/CodeForExp/JT_Semantomotor/data/bids/results/first-level/sub-04/sub-04_task-gesture_run-03_desc-concrete_effect_size.nii.gz


/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'jitter.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'ms_txt' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'trial_fixation.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'gesture_image' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/

/Users/jonathantsay/GestureContextPilot/CodeForExp/JT_Semantomotor/data/bids/results/first-level/sub-04/sub-04_task-gesture_run-04_desc-concrete_effect_size.nii.gz


/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'jitter.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'ms_txt' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'trial_fixation.started' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:88: UserWarning: Unexpected column 'gesture_image' in events data. It will be ignored.
  warnings.warn(
/Users/jonathantsay/anaconda3/envs/WeatherModel/lib/python3.11/site-packages/

/Users/jonathantsay/GestureContextPilot/CodeForExp/JT_Semantomotor/data/bids/results/first-level/sub-04/sub-04_task-gesture_run-05_desc-concrete_effect_size.nii.gz


In [4]:
jason_file = os.path.join(ds_path, 'results', model_name, "dataset_description.json")

if not os.path.exists(jason_file):
    import json
    import datetime
    from importlib.metadata import version

    nilearn_version = version('nilearn')
    date_created = datetime.datetime.now()
    
    # Data to be written
    content = {
        "Name": "First-level GLM analysis",
        "BIDSVersion": "1.4.0",
        "DatasetType": "results",
        "GeneratedBy": [
            {
                "Name": "Nilearn",
                "Version": nilearn_version,
                "CodeURL": "https://nilearn.github.io"
            }
        ],    
        "Date": date_created,
        "FirstLevelModel": [
            fmri_glm.get_params()
        ], 
    }
    
    # Serializing json
    json_object = json.dumps(content, indent=4, default=str)
    
    # Writing to .json
    with open(jason_file, "w") as outfile:
        outfile.write(json_object)
        